### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let\'s start by recalling what I know about parrots. They\'re birds, right? And they\'re known for mimicking human speech. But why do they do that? Maybe it\'s related to their natural behavior? I remember that some birds use vocalizations to communicate with each other, like for attracting mates or defending territory. So maybe parrots mimic human speech as a way of social interaction?\n\nWait, but in the wild, parrots don\'t talk like humans. So when they\'re kept as pets, they learn to mimic humans. Is that because they\'re trying to communicate with their owners? Or is it just a behavior they pick up from being around people a lot? Also, some sources say that parrots have a good memory and can associate words with actions or objects. So maybe they learn to say words to get attention or rewards.\n\nAnother angle: parrots have a syrinx, which is their vocal organ, allowing them to produce a wide range of soun

In [2]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [3]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which is Boston here. I'll call the function with location set to Boston. Make sure the JSON is correctly formatted with the name and arguments.\n", 'tool_calls': [{'id': 'd8p5y6j3r', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 154, 'total_tokens': 240, 'completion_time': 0.148292429, 'completion_tokens_details': {'reasoning_tokens': 62}, 'prompt_time': 0.006287013, 'prompt_tokens_details': None, 'queue_time': 0.088392032, 'total_time': 0.154579442}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--019c733a-41ab-78d1-8631-41848a9bc8e6-0' tool_call

### Tool Execution Loops

In [4]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

<bound method BaseMessage.text of AIMessage(content='The weather in Boston is sunny today. It looks like a great day to be outdoors!', additional_kwargs={'reasoning_content': 'Okay, the user asked, "What\'s the weather in Boston?" I called the get_weather function with Boston as the location. The response came back saying it\'s sunny in Boston. Now I need to relay that information to the user in a natural way.\n\nFirst, I should start by directly answering the question. The user wants to know the weather, so I should state that clearly. The function response says "sunny," so I can say something like, "The weather in Boston is sunny today."\n\nI should check if there\'s any additional information I might need to include. The function response was straightforward, just stating the condition. There\'s no mention of temperature or other details, so I don\'t need to add anything else. \n\nI should make sure the response is friendly and concise. Maybe add a sentence about enjoying the weathe

In [7]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. The function requires the location parameter, which in this case is Boston. So I\'ll call the function with location set to "Boston".\n', 'tool_calls': [{'id': 'wytmvqjqp', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 153, 'total_tokens': 225, 'completion_time': 0.104309374, 'completion_tokens_details': {'reasoning_tokens': 48}, 'prompt_time': 0.007226839, 'prompt_tokens_details': None, 'queue_time': 0.089000746, 'total_time': 0.111536213}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--019c733b-179a-78f1-adb1-e1875fd40401-0', tool_c

In [6]:
len(messages)

3